<a href="https://colab.research.google.com/github/aguilarmadeira/CIO/blob/main/Exercicio2_FilasEspera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np

# Parâmetros do problema
lmbda = 18  # Taxa de chegada (clientes por hora)
mu = 15  # Taxa de serviço (clientes por hora)
custo_salario_por_hora = 20  # Custo do salário por hora por caixa
custo_espera_por_minuto = 0.25  # Custo de espera por cliente por minuto
custo_espera_por_hora = custo_espera_por_minuto * 60  # Converter para custo por hora

# Calcular o número mínimo de caixas para um sistema estável
s_min = math.ceil(lmbda / mu)
max_caixas = s_min + 5  # Considerar um número maior de caixas

# Inicializar arrays para armazenar os custos
custos_salarios = np.inf * np.ones(max_caixas)
custos_espera = np.inf * np.ones(max_caixas)
custos_totais = np.inf * np.ones(max_caixas)
P_wait = np.zeros(max_caixas)  # Array para armazenar P(j > s)

# Calcular custos para diferentes números de caixas abertas
for s in range(s_min, max_caixas + 1):
    rho = lmbda / (s * mu)
    if rho >= 1:
        continue  # O sistema não é estável se rho >= 1

    # Calcular P0 (probabilidade de não haver clientes no sistema)
    P0 = 1 / (sum((lmbda / mu)**n / math.factorial(n) for n in range(s)) + (lmbda / mu)**s / (math.factorial(s) * (1 - rho)))

    # Calcular P(j > s)
    P_wait[s-1] = ((lmbda / mu)**s / math.factorial(s)) * P0 / (1 - rho)

    # Calcular Lq usando P(j > s)
    Lq = (P_wait[s-1] * rho) / (1 - rho)

    # Custos
    custos_salarios[s-1] = s * custo_salario_por_hora
    custos_espera[s-1] = Lq * custo_espera_por_hora
    custos_totais[s-1] = custos_salarios[s-1] + custos_espera[s-1]

    # Exibir valores para cada s
    print(f's = {s}')
    print(f'  Custo de salários: {custos_salarios[s-1]:.2f} u.m.')
    print(f'  Custo de espera: {custos_espera[s-1]:.2f} u.m.')
    print(f'  Custo total: {custos_totais[s-1]:.2f} u.m.')
    print(f'  Probabilidade de um cliente ter que esperar (P(j > s)): {P_wait[s-1]:.4f}')

# Encontrar o número ótimo de caixas que minimiza o custo total
num_otimo_caixas = np.argmin(custos_totais[s_min-1:max_caixas]) + s_min

# Encontrar o número de caixas que garante P(j > s) < 0.05
s_satisfacao = np.where((P_wait < 0.05) & (custos_totais < np.inf))[0]
s_satisfacao = s_satisfacao[0] + 1 if len(s_satisfacao) > 0 else None

# Exibir resultados finais
print(f'\nNúmero mínimo de caixas para estabilidade: {s_min}')
print(f'Número ótimo de caixas para minimizar custo: {num_otimo_caixas}')
print(f'Custo total mínimo: {custos_totais[num_otimo_caixas-1]:.2f} u.m.')
print(f'Probabilidade de um cliente ter que esperar (P(j > s)) com {num_otimo_caixas} caixas: {P_wait[num_otimo_caixas-1]:.4f}')

if s_satisfacao is not None:
    print(f'Número de caixas para garantir P(j > s) < 0.05: {s_satisfacao}')
    print(f'Custo total nessa configuração: {custos_totais[s_satisfacao-1]:.2f} u.m.')
else:
    print(f'Não foi possível garantir P(j > s) < 0.05 com até {max_caixas} caixas.')
